# Detecting Bots in Covid-19 Tweets

In [31]:
from IPython.display import HTML, display
import tabulate

In [1]:
%run ../aux/postgres.py

postgres functions loaded...


## Sets of Tweets in Portuguese and French

In [53]:
table = query("select codamostra, descricao, (select count(at.codtweet) from amostratweet at where at.codamostra = a.codamostra) from amostra a where lower(descricao) like '%covid%'")
display(HTML(tabulate.tabulate(table, tablefmt='html')))

222,Covid-19 PT,12865843
223,Covid-19 FR,11861867


## Exporting tweets to csv:

~~~~
psql -d twitter -t -c "select json from tweet where codprojeto = 37 and datatweet between '2020-03-20' and '2020-04-09' and json->>'lang' = 'pt'"  > tweetspt.txt

psql -d twitter -t -c "select json from tweet where codprojeto = 37 and datatweet between '2020-03-20' and '2020-04-09' and json->>'lang' = 'fr'"  > tweetsfr.txt

mateus@kode:~/projetos/absa/20_corona/data$ wc -l tweetspt.txt
12.865.844 tweetspt.txt

mateus@kode:~/projetos/absa/20_corona/data$ wc -l tweetsfr.txt
11.861.868 tweetsfr.txt
~~~~

## SQL sets of tweets in Portuguese and French:

~~~~sql
INSERT INTO amostratweet (codamostra, codtweet) select 222, codtweet from tweet where codprojeto = 37 and datatweet between '2020-03-20' and '2020-04-09' and json->>'lang' = 'pt';

insert into amostratweet (codamostra, codtweet) select 223, codtweet from tweet where codprojeto = 37 and datatweet between '2020-03-20' and '2020-04-09' and json->>'lang' = 'fr';
~~~~

## Rule 01: users with more than 100 tweets in a single day

~~~~
psql -d twitter -t -c "SELECT userid, datatweet, count(t.codtweet) as tweets FROM tweet t INNER JOIN amostratweet at ON at.codtweet = t.codtweet WHERE at.codamostra = 222 GROUP BY userid, datatweet" > tweets_user_day_pt.csv

psql -d twitter -t -c "SELECT userid, datatweet, count(t.codtweet) as tweets FROM tweet t INNER JOIN amostratweet at ON at.codtweet = t.codtweet WHERE at.codamostra = 223 GROUP BY userid, datatweet" > tweets_user_day_fr.csv

~~~~


~~~~sql
insert into deteccaobotuserid (coddeteccaobot, userid) SELECT distinct 1, userid FROM tweet t INNER JOIN amostratweet at ON at.codtweet = t.codtweet WHERE at.codamostra = 222 GROUP BY userid, datatweet having count(t.codtweet) >= 100;
insert into deteccaobotuserid (coddeteccaobot, userid) SELECT distinct 2, userid FROM tweet t INNER JOIN amostratweet at ON at.codtweet = t.codtweet WHERE at.codamostra = 223 GROUP BY userid, datatweet having count(t.codtweet) >= 100;
~~~~

In [37]:
table = query("SELECT dbu.coddeteccaobot, db.descricao, count(*) as users FROM deteccaobotuserid dbu INNER JOIN deteccaobot db ON db.coddeteccaobot = dbu.coddeteccaobot WHERE db.descricao like 'Tweets por dia%' GROUP BY 1, 2")
display(HTML(tabulate.tabulate(table, tablefmt='html')))

1,Tweets por dia pt,297
2,Tweets por dia fr,653


## Rule 02: user without profile image

222 - português
223 - francês
~~~~sql
insert into deteccaobotuserid (coddeteccaobot, userid) SELECT distinct 4, userid FROM tweet t INNER JOIN amostratweet at ON at.codtweet = t.codtweet WHERE at.codamostra = 222 AND json->'user'->>'default_profile_image' = 'false';
insert into deteccaobotuserid (coddeteccaobot, userid) SELECT distinct 5, userid FROM tweet t INNER JOIN amostratweet at ON at.codtweet = t.codtweet WHERE at.codamostra = 223 AND json->'user'->>'default_profile_image' = 'false';

~~~~

In [40]:
table = query("SELECT dbu.coddeteccaobot, db.descricao, count(*) as users FROM deteccaobotuserid dbu INNER JOIN deteccaobot db ON db.coddeteccaobot = dbu.coddeteccaobot WHERE db.descricao like 'Default Profile Image%' GROUP BY 1, 2")
display(HTML(tabulate.tabulate(table, tablefmt='html')))

4,Default Profile Image pt,2203618
5,Default Profile Image fr,1340381


## Rule 03: followers count

~~~~sql
--- followers count
insert into deteccaobotuserid (coddeteccaobot, userid) SELECT distinct 6, userid FROM tweet t INNER JOIN amostratweet at ON at.codtweet = t.codtweet WHERE t.codprojeto = 37 and at.codamostra = 222 AND json->'user'->>'followers_count' = '0';

insert into deteccaobotuserid (coddeteccaobot, userid) SELECT distinct 7, userid FROM tweet t INNER JOIN amostratweet at ON at.codtweet = t.codtweet WHERE t.codprojeto = 37 and at.codamostra = 223 AND json->'user'->>'followers_count' = '0';  
~~~~

In [39]:
table = query("SELECT dbu.coddeteccaobot, db.descricao, count(*) as users FROM deteccaobotuserid dbu INNER JOIN deteccaobot db ON db.coddeteccaobot = dbu.coddeteccaobot WHERE db.descricao like 'Followers%' GROUP BY 1, 2")
display(HTML(tabulate.tabulate(table, tablefmt='html')))

6,Followers count pt,38462
7,Followers count fr,35446


## Rule 04: friends count

~~~~sql
--- friends count
insert into deteccaobotuserid (coddeteccaobot, userid) SELECT distinct 8, userid FROM tweet t INNER JOIN amostratweet at ON at.codtweet = t.codtweet WHERE t.codprojeto = 37 and at.codamostra = 222 AND json->'user'->>'friends_count' = '0';

insert into deteccaobotuserid (coddeteccaobot, userid) SELECT distinct 9, userid FROM tweet t INNER JOIN amostratweet at ON at.codtweet = t.codtweet WHERE t.codprojeto = 37 and at.codamostra = 223 AND json->'user'->>'friends_count' = '0';  
~~~~

In [41]:
table = query("SELECT dbu.coddeteccaobot, db.descricao, count(*) as users FROM deteccaobotuserid dbu INNER JOIN deteccaobot db ON db.coddeteccaobot = dbu.coddeteccaobot WHERE db.descricao like 'Friends%' GROUP BY 1, 2")
display(HTML(tabulate.tabulate(table, tablefmt='html')))

8,Friends count pt,9034
9,Friends count fr,10138


## Rule 05: difference betweet user creation date and tweet creation date

~~~~sql
--- created at
insert into deteccaobotuserid (coddeteccaobot, userid) SELECT distinct 10, userid FROM tweet t INNER JOIN amostratweet at ON at.codtweet = t.codtweet WHERE t.codprojeto = 37 and at.codamostra = 222 AND ((json->>'created_at')::date - (json->'user'->>'created_at')::date) <=7;

insert into deteccaobotuserid (coddeteccaobot, userid) SELECT distinct 11, userid FROM tweet t INNER JOIN amostratweet at ON at.codtweet = t.codtweet WHERE t.codprojeto = 37 and at.codamostra = 223 AND ((json->>'created_at')::date - (json->'user'->>'created_at')::date) <= 7;  
~~~~

In [42]:
table = query("SELECT dbu.coddeteccaobot, db.descricao, count(*) as users FROM deteccaobotuserid dbu INNER JOIN deteccaobot db ON db.coddeteccaobot = dbu.coddeteccaobot WHERE db.descricao like 'Create%' GROUP BY 1, 2")
display(HTML(tabulate.tabulate(table, tablefmt='html')))

10,Created at pt,57924
11,Created at fr,34767


# Rules Combination

## Users matching more than 3 rules
~~~~sql
insert into deteccaobotuserid (coddeteccaobot, userid) SELECT 12, userid from deteccaobotuserid where coddeteccaobot in (1, 4, 6, 8, 10) group by userid having count(*) >= 3;

insert into deteccaobotuserid (coddeteccaobot, userid) SELECT 14, userid from deteccaobotuserid where coddeteccaobot in (2, 5, 7, 9, 11) group by userid having count(*) >= 3;
~~~~

In [51]:
table = query("SELECT dbu.coddeteccaobot, db.descricao, count(*) as users FROM deteccaobotuserid dbu INNER JOIN deteccaobot db ON db.coddeteccaobot = dbu.coddeteccaobot WHERE db.descricao like 'Group by 3%' GROUP BY 1, 2")
display(HTML(tabulate.tabulate(table, tablefmt='html')))

12,Group by 3 pt,19452
14,Group by 3 fr,17430


## Users matching more than 4 rules
~~~~sql
insert into deteccaobotuserid (coddeteccaobot, userid) SELECT 13, userid from deteccaobotuserid where coddeteccaobot in (1, 4, 6, 8, 10) group by userid having count(*) >= 4;

insert into deteccaobotuserid (coddeteccaobot, userid) SELECT 15, userid from deteccaobotuserid where coddeteccaobot in (2, 5, 7, 9, 11) group by userid having count(*) >= 4;
~~~~

In [52]:
table = query("SELECT dbu.coddeteccaobot, db.descricao, count(*) as users FROM deteccaobotuserid dbu INNER JOIN deteccaobot db ON db.coddeteccaobot = dbu.coddeteccaobot WHERE db.descricao like 'Group by 4%' GROUP BY 1, 2")
display(HTML(tabulate.tabulate(table, tablefmt='html')))

13,Group by 4 pt,2983
15,Group by 4 fr,3721
